# Setup

## Imports

In [4]:
import QuantumOptics as qo
using Logging

using LinearAlgebra
using SparseArrays
import CairoMakie as cm

using ProtoStructs

import QuantumOptics.⊗
import QuantumOptics.*

import CSV
using JSON
import Tables

using YAXArrays

using MiniLoggers
using Dates
using Revise

In [5]:
import SuperconductingCavities as SC

## Some Extra Stuff

In [6]:
# MiniLogger(minlevel = MiniLoggers.Info) |> global_logger
InfoLogger = MiniLogger(minlevel = MiniLoggers.Info)
ProgressLogger = MiniLogger(minlevel = LogLevel(-1))
DebugLogger = MiniLogger(minlevel = MiniLoggers.Debug)

global_logger(ProgressLogger)

In [ ]:
function tostr(obj)
    io = IOBuffer()
    show(io, "text/plain", obj)
    String(take!(io))
end

# Example

## Initializing Models

### Creating Model From Scratch

#### One Mode

In [11]:
Eᶜ = 0.10283303447280807;
Eʲ = 26.96976142643705;
Eᵒˢᶜ = 6.2280871213;
g = 0.026184375000000013;
name = "transmon";
ng  =  0;
N=10;
N_cut = 60;

OneMode = SC.Transmon_Resonators_Constructor(Eᶜ, Eʲ, [Eᵒˢᶜ], [g], N, [N], Cavity_Names = ["Mode 3"]);

#### Two Mode

In [12]:
Eᶜ = 0.10283303447280807;
Eʲ = 26.96976142643705;
Eᵒˢᶜs = [6.2280871213, 6.72062927579];
gs = [0.026184375000000013, 0.03020625000000001];
name = "transmon";
ng  =  0;
Nₜ =10;
Nᵣs = [6, 6]
N_cut = 60;


TwoModes = SC.Transmon_Resonators_Constructor(Eᶜ, Eʲ, Eᵒˢᶜs, gs, Nₜ, Nᵣs, Cavity_Names = ["Mode 3", "Mode 5"]);

### Importing Model

In [ ]:
Mode3 = SC.Transmon_Resonators_Loader("ModelSaves/Mode_3/Mode_3.json");

In [ ]:
Mode3.Stuff["op_drive_params"]["sb_f_0_Main"]

## Time Evolving Wavefunctions

### Running A Single Pulse

In [21]:
ψ = Mode3.dressed_states[(0,0)]
SC.RunSingleOperator(Mode3, ψ, Mode3.Stuff["op_drive_params"]["q_g_0"], save_step = true, to_return = "Nothing", step_name = "q_g_0", run_name = "Single_Op_Demo_"*string(now()));

[2024-09-01 18:02:32] Info: Running Time Evolution
[2024-09-01 18:02:32] LogLevel(-1): ODE progress = 0
[2024-09-01 18:02:32] LogLevel(-1): ODE message = dt=0.0010432581424918218	t=1.0419916278610004	max u=-0.4512538658823686 - 0.8923725857046029im, progress = 0.010399118042524955
[2024-09-01 18:02:33] LogLevel(-1): ODE message = dt=0.0010432591097843593	t=2.0852500959849563	max u=-0.7382805949277542 + 0.674420209921304im, progress = 0.02081087920144667
[2024-09-01 18:02:34] LogLevel(-1): ODE message = dt=0.001043258764323398	t=3.128509125898927	max u=0.8479033027523399 + 0.5299575367499055im, progress = 0.03122264596705516
[2024-09-01 18:02:35] LogLevel(-1): ODE message = dt=0.0010432606298179875	t=4.171768857930078	max u=0.2827994955471448 - 0.9589923961793032im, progress = 0.04163441973982114
[2024-09-01 18:02:35] LogLevel(-1): ODE message = dt=0.001043261182558991	t=5.215029448988226	max u=-0.9995964417054138 - 0.015046979091596932im, progress = 0.052046202085710834
[2024-09-01 18:

### Running a Series of Operators

In [12]:
ψ = Mode3.dressed_states[(0,0)]
SC.RunPulseSequence(Mode3, ψ, ["q_g_0", "q_e_0"])

UndefVarError: UndefVarError: `model` not defined

## With Density Matrices

In [63]:
Mode3.CandD_Ops

Dict{Any, Any} with 3 entries:
  "Mode 3 Collapse"    => Operator(dim=100x100)…
  "Transmon Collapse"  => Operator(dim=100x100)…
  "Transmon Dephasing" => Operator(dim=100x100)…

In [64]:
ψ = Mode3.dressed_states[(0,0)]
ρ = ψ*ψ'
SC.RunPulseSequence(Mode3, ρ, ["q_g_0", "q_e_0", "sb_f_0_Main"], c_ops = collect(values(Mode3.CandD_Ops)), run_name = "Demo_Run_With_With_Loss_"*string(now()))

[2024-09-01 20:32:54] Info: Running operator q_g_0
[2024-09-01 20:32:54] Info: Running Time Evolution
[2024-09-01 20:32:54] LogLevel(-1): ODE progress = 0
[2024-09-01 20:33:24] LogLevel(-1): ODE message = dt=0.002230153921423261	t=2.5717702130572	max u=0.9998799958948087 + 9.043977369536986e-15im, progress = 0.025666369391788426
[2024-09-01 20:33:52] LogLevel(-1): ODE message = dt=0.0020840758645687164	t=4.71829307410324	max u=0.9995778846577051 + 1.3497386220177304e-14im, progress = 0.047088753234563276
[2024-09-01 20:34:20] LogLevel(-1): ODE message = dt=0.00201073384014774	t=6.762436886864862	max u=0.9990106511042884 + 1.2611209794102074e-14im, progress = 0.06748939008847167
[2024-09-01 20:35:19] LogLevel(-1): ODE message = dt=0.0019642733022675377	t=8.748407711717487	max u=0.9980731959355491 + 1.2656372087897826e-14im, progress = 0.08730945820077332
[2024-09-01 20:36:06] LogLevel(-1): ODE message = dt=0.0019315465604094616	t=10.695442233532683	max u=0.9966841858216658 + 1.210160895

InterruptException: InterruptException:

In [50]:
dat_nl = open_dataset("Data/Demo_Run_With_No_Loss_2024-09-01T19:29:42.289.nc")
dat_wl = open_dataset("Data/Demo_Run_With_Collapse_and_Dephasing_2024-09-01T18:59:23.478.nc")

YAXArray Dataset
Shared Axes: 
(↓ State Categorical{String} [(7, 1), (0, 3), …, (9, 0), (6, 7)] Unordered)
Variables: 
Step_1
  (↓ Step_1_Steps Sampled{Int64} 0:1:501 ForwardOrdered Regular Points)
Step_2
  (↓ Step_2_Steps Sampled{Int64} 0:1:358 ForwardOrdered Regular Points)
Step_3
  (↓ Step_3_Steps Sampled{Int64} 0:1:889 ForwardOrdered Regular Points)


In [51]:
println(dat_nl.Step_3[At.([string((0,1)), 889])...])
println(dat_wl.Step_3[At.([string((0,1)), 889])...])

0.9952291721177998
0.9952291721177998


In [52]:
println(dat_nl.Step_2[At.([string((2,0)), 358])...])
println(dat_wl.Step_2[At.([string((2,0)), 358])...])

0.9959011012603762
0.9959011012603762


In [53]:
println(dat_nl.Step_1[At.([string((1,0)), 501])...])
println(dat_wl.Step_1[At.([string((1,0)), 501])...])

0.9982731436728601
0.9982731436728601
